# Basic Transformations

## Overview of Basic Transformations

Let us define problem statements to learn more about Data Frame APIs. We will try to cover filtering, aggregations and sorting as part of solutions for these problem statements.
* Get total number of flights as well as number of flights which are delayed in departure and number of flights delayed in arrival. 
  * Output should contain 3 columns - **FlightCount**, **DepDelayedCount**, **ArrDelayedCount**
* Get number of flights which are delayed in departure and number of flights delayed in arrival for each day along with number of flights departed for each day. 
  * Output should contain 4 columns - **FlightDate**, **FlightCount**, **DepDelayedCount**, **ArrDelayedCount**
  * **FlightDate** should be of **YYYY-MM-dd** format.
  * Data should be **sorted** in ascending order by **flightDate**
* Get all the flights which are departed late but arrived early (**IsArrDelayed is NO**).
  * Output should contain - **FlightCRSDepTime**, **UniqueCarrier**, **FlightNum**, **Origin**, **Dest**, **DepDelay**, **ArrDelay**
  * **FlightCRSDepTime** need to be computed using **Year**, **Month**, **DayOfMonth**, **CRSDepTime**
  * **FlightCRSDepTime** should be displayed using **YYYY-MM-dd HH:mm** format.
  * Output should be sorted by **FlightCRSDepTime** and then by the difference between **DepDelay** and **ArrDelay**
  * Also get the count of such flights

## Starting Spark Context

Let us start spark context for this Notebook so that we can execute the code provided.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession. \
    builder. \
    config("spark.ui.port", "0"). \
    enableHiveSupport(). \
    appName("Processing Column Data"). \
    master("yarn"). \
    getOrCreate()

 
## Overview of Filtering
Let us understand few important details related to filtering before we get into the solution
* Filtering can be done either by using `filter` or `>where`. These are like synonyms to each other.
* When it comes to the condition, we can either pass it in **SQL Style** or **Data Frame Style**.
* Example for SQL Style - `airlines.filter("IsArrDelayed = 'YES'").show() or airlines.where("IsArrDelayed = 'YES'").show()`
* Example for Data Frame Style - `airlines.filter(airlines["IsArrDelayed"] == 'YES').show()</mark> or <mark>airlines.filter(airlines.IsArrDelayed == 'YES').show()`. We can also use where instead of filter.
* Here are the other operations we can perform to filter the data - `!=`, `>`, `<`>, `>=`, `<=`, `LIKE`, `BETWEEN` with `AND`
* If we have to validate against multiple columns then we need to use boolean operations such as `AND` and `OR`.
* If we have to compare each column value with multiple values then we can use the `IN` operator.
    

### Tasks

Let us perform some tasks to understand filtering in detail. Solve all the problems by passing  conditions using both SQL Style as well as API Style.

* Read the data for the month of 2008 January.

In [2]:
airlines_path = "/public/airlines_all/airlines-part/flightmonth=200801"

In [3]:
airlines = spark. \
    read. \
    parquet(airlines_path)

In [4]:
airlines.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Car

* Get count of flights which are departed late at origin and reach destination early or on time.


In [5]:
airlines. \
    filter("IsDepDelayed = 'YES' AND IsArrDelayed = 'NO'"). \
    count()

54233

* API Style

In [6]:
from pyspark.sql.functions import col

In [7]:
airlines. \
    filter((col("IsDepDelayed") == "YES") & 
           (col("IsArrDelayed") == "NO")
          ). \
    count()

54233

In [8]:
airlines. \
    filter((airlines["IsDepDelayed"] == "YES") & 
           (airlines.IsArrDelayed == "NO")
          ). \
    count()

54233

* Get count of flights which are departed late from origin by more than 60 minutes.


In [9]:
airlines. \
    filter("DepDelay > 60"). \
    count()

40104


* API Style

In [10]:
from pyspark.sql.functions import col

airlines. \
    filter(col("DepDelay") > 60). \
    count()

40104

* Get count of flights which are departed early or on time but arrive late by at least 15 minutes.


In [11]:
airlines. \
    filter("IsDepDelayed = 'NO' AND ArrDelay >= 15"). \
    count()

20705

* API Style

In [12]:
from pyspark.sql.functions import col

airlines. \
    filter((col("IsDepDelayed") == "NO") & (col("ArrDelay") >= 15)). \
    count()

20705

* Get count of flights departed from following major airports - ORD, DFW, ATL, LAX, SFO.

In [13]:
airlines. \
    filter("Origin IN ('ORD', 'DFW', 'ATL', 'LAX', 'SFO')"). \
    count()

118212

In [14]:
airlines.count()

605659

* API Style

In [15]:
from pyspark.sql.functions import col
c = col('x')
help(c.isin)

Help on method isin in module pyspark.sql.column:

isin(*cols) method of pyspark.sql.column.Column instance
    A boolean expression that is evaluated to true if the value of this
    expression is contained by the evaluated values of the arguments.
    
    >>> df[df.name.isin("Bob", "Mike")].collect()
    [Row(age=5, name='Bob')]
    >>> df[df.age.isin([1, 2, 3])].collect()
    [Row(age=2, name='Alice')]
    
    .. versionadded:: 1.5



In [16]:
from pyspark.sql.functions import col

airlines. \
    filter(col("Origin").isin("ORD", "DFW", "ATL", "LAX", "SFO")). \
    count()

118212

* Add a column FlightDate by using Year, Month and DayOfMonth. Format should be **yyyyMMdd**.


In [17]:
from pyspark.sql.functions import col, concat, lpad

airlines. \
    withColumn("FlightDate",
                concat(col("Year"),
                       lpad(col("Month"), 2, "0"),
                       lpad(col("DayOfMonth"), 2, "0")
                      )
              ). \
    show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+----------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|IsArrDelayed|IsDepDelayed|FlightDate|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+--------

* Get count of flights departed late between 2008 January 1st to January 9th using FlightDate.


In [18]:
from pyspark.sql.functions import col, concat, lpad

airlines. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter("IsDepDelayed = 'YES' AND FlightDate LIKE '2008010%'"). \
    count()

91045

In [19]:
from pyspark.sql.functions import col, concat, lpad

airlines. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter("""
           IsDepDelayed = 'YES' AND 
           FlightDate BETWEEN 20080101 AND 20080109
          """). \
    count()

91045

* API Style

In [20]:
from pyspark.sql.functions import col
c = col('x')
help(c.like)

Help on method _ in module pyspark.sql.column:

_(other) method of pyspark.sql.column.Column instance
    SQL like expression. Returns a boolean :class:`Column` based on a SQL LIKE match.
    
    :param other: a SQL LIKE pattern
    
    See :func:`rlike` for a regex version
    
    >>> df.filter(df.name.like('Al%')).collect()
    [Row(age=2, name='Alice')]



In [21]:
from pyspark.sql.functions import col, concat, lpad

airlines. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter((col("IsDepDelayed") == "YES") & 
           (col("FlightDate").like("2008010%"))
          ). \
    count()

91045

In [22]:
from pyspark.sql.functions import col
c = col('x')
help(c.between)

Help on method between in module pyspark.sql.column:

between(lowerBound, upperBound) method of pyspark.sql.column.Column instance
    A boolean expression that is evaluated to true if the value of this
    expression is between the given columns.
    
    >>> df.select(df.name, df.age.between(2, 4)).show()
    +-----+---------------------------+
    | name|((age >= 2) AND (age <= 4))|
    +-----+---------------------------+
    |Alice|                       true|
    |  Bob|                      false|
    +-----+---------------------------+
    
    .. versionadded:: 1.3



In [23]:
from pyspark.sql.functions import col, concat, lpad

airlines. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter((col("IsDepDelayed") == "YES") & 
           (col("FlightDate").between("20080101", "20080109"))
          ). \
    count()

91045

* Get number of flights departed late on Sundays.

In [24]:
l = [('X',)]
df = spark.createDataFrame(l, "dummy STRING")

In [25]:
from pyspark.sql.functions import current_date
df.select(current_date()).show()

+--------------+
|current_date()|
+--------------+
|    2023-08-01|
+--------------+



In [26]:
from pyspark.sql.functions import date_format

df.select(current_date(), date_format(current_date(), 'EE')).show()

+--------------+-------------------------------+
|current_date()|date_format(current_date(), EE)|
+--------------+-------------------------------+
|    2023-08-01|                            Tue|
+--------------+-------------------------------+



In [27]:
from pyspark.sql.functions import col, concat, lpad

airlines. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter("""
           IsDepDelayed = 'YES' AND 
           date_format(to_date(FlightDate, 'yyyyMMdd'), 'EEEE') = 'Sunday'
           """). \
    count()

34708

* API Style

In [28]:
from pyspark.sql.functions import col, concat, lpad, date_format, to_date

airlines. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter((col("IsDepDelayed") == "YES") &
           (date_format(
               to_date("FlightDate", "yyyyMMdd"), "EEEE"
           ) == "Sunday")
          ). \
    count()

34708

## Overview of Aggregations

Let us go through the details related to aggregation using Spark.

* We can perform total aggregations directly on Dataframe or we can perform aggregations after grouping by a key(s).
* Here are the APIs which we typically use to group the data using a key.
  * groupBy
  * rollup
  * cube
* Here are the functions which we typically use to perform aggregations.
  * count
  * sum, avg
  * min, max
* If we want to provide aliases to the aggregated fields then we have to use `agg` after `groupBy`.

## Overview of Sorting

Let us understand how to sort the data in a Data Frame.
* We can use `orderBy` or `sort` to sort the data.
* We can perform composite sorting by passing multiple columns or expressions.
* By default data is sorted in ascending order, we can change it to descending by applying `desc()` function on the column or expression.

## Solutions - Problem 1
Get total number of flights as well as number of flights which are delayed in departure and number of flights delayed in arrival. 
* Output should contain 3 columns - **FlightCount**, **DepDelayedCount**, **ArrDelayedCount**

### Reading airlines data

In [29]:
airlines_path = "/public/airlines_all/airlines-part/flightmonth=200801"

airlines = spark. \
    read. \
    parquet(airlines_path)

airlines.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Car

### Get flights with delayed arrival

In [30]:
# SQL Style
airlines.filter("IsArrDelayed = 'YES'").show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|IsArrDelayed|IsDepDelayed|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
|2008|    1|    

In [31]:
# Data Frame Style
airlines.filter(airlines["IsArrDelayed"] == 'YES').show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|IsArrDelayed|IsDepDelayed|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
|2008|    1|    

In [32]:
airlines.filter(airlines.IsArrDelayed == 'YES').show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|IsArrDelayed|IsDepDelayed|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
|2008|    1|    

### Get delayed counts

In [33]:
## Departure Delayed Count
airlines. \
    filter(airlines.IsDepDelayed == "YES"). \
    count()

265198

In [34]:
## Arrival Delayed Count
airlines. \
    filter(airlines.IsArrDelayed == "YES"). \
    count()

297956

In [35]:
airlines. \
    filter("IsDepDelayed = 'YES' OR IsArrDelayed = 'YES'"). \
    select('Year', 'Month', 'DayOfMonth', 
           'FlightNum', 'IsDepDelayed', 'IsArrDelayed'
          ). \
    show()

+----+-----+----------+---------+------------+------------+
|Year|Month|DayOfMonth|FlightNum|IsDepDelayed|IsArrDelayed|
+----+-----+----------+---------+------------+------------+
|2008|    1|        17|     4977|         YES|         YES|
|2008|    1|        17|     5426|          NO|         YES|
|2008|    1|        18|     5260|         YES|         YES|
|2008|    1|        19|     5276|          NO|         YES|
|2008|    1|        20|     5215|          NO|         YES|
|2008|    1|        20|     5324|          NO|         YES|
|2008|    1|        21|     5056|          NO|         YES|
|2008|    1|        21|     5215|         YES|          NO|
|2008|    1|        21|     5595|         YES|         YES|
|2008|    1|        21|     5610|          NO|         YES|
|2008|    1|        22|     5032|         YES|         YES|
|2008|    1|        22|     5331|          NO|         YES|
|2008|    1|        23|     5033|          NO|         YES|
|2008|    1|        23|     5355|       

In [36]:
## Both Departure Delayed and Arrival Delayed
from pyspark.sql.functions import col, lit, count, sum, expr
airlines. \
    agg(count(lit(1)).alias("FlightCount"),
        sum(expr("CASE WHEN IsDepDelayed = 'YES' THEN 1 ELSE 0 END")).alias("DepDelayedCount"),
        sum(expr("CASE WHEN IsArrDelayed = 'YES' THEN 1 ELSE 0 END")).alias("ArrDelayedCount")
       ). \
    show()

+-----------+---------------+---------------+
|FlightCount|DepDelayedCount|ArrDelayedCount|
+-----------+---------------+---------------+
|     605659|         265198|         297956|
+-----------+---------------+---------------+



## Solutions - Problem 2

Get number of flights which are delayed in departure and number of flights delayed in arrival for each day along with number of flights departed for each day. 

* Output should contain 4 columns - **FlightDate**, **FlightCount**, **DepDelayedCount**, **ArrDelayedCount**
* **FlightDate** should be of **YYYY-MM-dd** format.
*   Data should be **sorted** in ascending order by **flightDate**

### Grouping Data by Flight Date


In [37]:
from pyspark.sql.functions import lit, concat, lpad
airlines. \
  groupBy(concat("Year", lit("-"), 
                 lpad("Month", 2, "0"), lit("-"), 
                 lpad("DayOfMonth", 2, "0")).
          alias("FlightDate"))

### Getting Counts by FlightDate

In [38]:
from pyspark.sql.functions import lit, concat, lpad, count

airlines. \
    groupBy(concat("Year", lit("-"), 
                   lpad("Month", 2, "0"), lit("-"), 
                   lpad("DayOfMonth", 2, "0")).
            alias("FlightDate")). \
    agg(count(lit(1)).alias("FlightCount")). \
    show(31)

+----------+-----------+
|FlightDate|FlightCount|
+----------+-----------+
|2008-01-15|      19503|
|2008-01-21|      20133|
|2008-01-11|      20349|
|2008-01-19|      16249|
|2008-01-02|      20953|
|2008-01-06|      19893|
|2008-01-29|      19485|
|2008-01-30|      19766|
|2008-01-17|      20273|
|2008-01-31|      20260|
|2008-01-01|      19175|
|2008-01-24|      20257|
|2008-01-07|      20341|
|2008-01-09|      19820|
|2008-01-04|      20929|
|2008-01-08|      19603|
|2008-01-05|      18066|
|2008-01-25|      20313|
|2008-01-26|      16276|
|2008-01-12|      16572|
|2008-01-16|      19764|
|2008-01-23|      19769|
|2008-01-18|      20347|
|2008-01-22|      19504|
|2008-01-28|      20147|
|2008-01-13|      18946|
|2008-01-10|      20297|
|2008-01-14|      20176|
|2008-01-20|      18653|
|2008-01-27|      18903|
|2008-01-03|      20937|
+----------+-----------+



In [39]:
# Alternative to get the count with out using agg
# We will not be able to provide alias for aggregated fields
from pyspark.sql.functions import lit, concat, lpad

airlines. \
    groupBy(concat("Year", lit("-"), 
                   lpad("Month", 2, "0"), lit("-"), 
                   lpad("DayOfMonth", 2, "0")).
            alias("FlightDate")). \
    count(). \
    show()

+----------+-----+
|FlightDate|count|
+----------+-----+
|2008-01-15|19503|
|2008-01-21|20133|
|2008-01-11|20349|
|2008-01-19|16249|
|2008-01-02|20953|
|2008-01-06|19893|
|2008-01-29|19485|
|2008-01-30|19766|
|2008-01-17|20273|
|2008-01-31|20260|
|2008-01-01|19175|
|2008-01-24|20257|
|2008-01-07|20341|
|2008-01-09|19820|
|2008-01-04|20929|
|2008-01-08|19603|
|2008-01-05|18066|
|2008-01-25|20313|
|2008-01-26|16276|
|2008-01-12|16572|
+----------+-----+
only showing top 20 rows



### Getting total as well as delayed counts for each day

In [40]:
from pyspark.sql.functions import lit, concat, lpad, count, sum, expr

airlines. \
    groupBy(concat("Year", lit("-"), 
                   lpad("Month", 2, "0"), lit("-"), 
                   lpad("DayOfMonth", 2, "0")).
            alias("FlightDate")). \
    agg(count(lit(1)).alias("FlightCount"),
        sum(expr("CASE WHEN IsDepDelayed = 'YES' THEN 1 ELSE 0 END")).alias("DepDelayedCount"),
        sum(expr("CASE WHEN IsArrDelayed = 'YES' THEN 1 ELSE 0 END")).alias("ArrDelayedCount")
       ). \
    show()

+----------+-----------+---------------+---------------+
|FlightDate|FlightCount|DepDelayedCount|ArrDelayedCount|
+----------+-----------+---------------+---------------+
|2008-01-15|      19503|           5603|           6688|
|2008-01-21|      20133|          10530|          11507|
|2008-01-11|      20349|           7779|           8688|
|2008-01-19|      16249|           7275|           7686|
|2008-01-02|      20953|          13805|          14260|
|2008-01-06|      19893|          11225|          11388|
|2008-01-29|      19485|           7213|           9259|
|2008-01-30|      19766|           7349|           8508|
|2008-01-17|      20273|          10507|          12101|
|2008-01-31|      20260|          10208|          12385|
|2008-01-01|      19175|          11053|          11725|
|2008-01-24|      20257|           8434|          10294|
|2008-01-07|      20341|           8701|           9262|
|2008-01-09|      19820|           6339|           7234|
|2008-01-04|      20929|       

### Sorting Data By FlightDate

In [41]:
help(airlines.sort)

Help on method sort in module pyspark.sql.dataframe:

sort(*cols, **kwargs) method of pyspark.sql.dataframe.DataFrame instance
    Returns a new :class:`DataFrame` sorted by the specified column(s).
    
    :param cols: list of :class:`Column` or column names to sort by.
    :param ascending: boolean or list of boolean (default ``True``).
        Sort ascending vs. descending. Specify list for multiple sort orders.
        If a list is specified, length of the list must equal length of the `cols`.
    
    >>> df.sort(df.age.desc()).collect()
    [Row(age=5, name='Bob'), Row(age=2, name='Alice')]
    >>> df.sort("age", ascending=False).collect()
    [Row(age=5, name='Bob'), Row(age=2, name='Alice')]
    >>> df.orderBy(df.age.desc()).collect()
    [Row(age=5, name='Bob'), Row(age=2, name='Alice')]
    >>> from pyspark.sql.functions import *
    >>> df.sort(asc("age")).collect()
    [Row(age=2, name='Alice'), Row(age=5, name='Bob')]
    >>> df.orderBy(desc("age"), "name").collect()
    

In [42]:
help(airlines.orderBy)

Help on method sort in module pyspark.sql.dataframe:

sort(*cols, **kwargs) method of pyspark.sql.dataframe.DataFrame instance
    Returns a new :class:`DataFrame` sorted by the specified column(s).
    
    :param cols: list of :class:`Column` or column names to sort by.
    :param ascending: boolean or list of boolean (default ``True``).
        Sort ascending vs. descending. Specify list for multiple sort orders.
        If a list is specified, length of the list must equal length of the `cols`.
    
    >>> df.sort(df.age.desc()).collect()
    [Row(age=5, name='Bob'), Row(age=2, name='Alice')]
    >>> df.sort("age", ascending=False).collect()
    [Row(age=5, name='Bob'), Row(age=2, name='Alice')]
    >>> df.orderBy(df.age.desc()).collect()
    [Row(age=5, name='Bob'), Row(age=2, name='Alice')]
    >>> from pyspark.sql.functions import *
    >>> df.sort(asc("age")).collect()
    [Row(age=2, name='Alice'), Row(age=5, name='Bob')]
    >>> df.orderBy(desc("age"), "name").collect()
    

In [43]:
from pyspark.sql.functions import lit, concat, lpad, sum, expr
airlines. \
    groupBy(concat("Year", lit("-"), 
                   lpad("Month", 2, "0"), lit("-"), 
                   lpad("DayOfMonth", 2, "0")).
            alias("FlightDate")). \
    agg(count(lit(1)).alias("FlightCount"),
        sum(expr("CASE WHEN IsDepDelayed = 'YES' THEN 1 ELSE 0 END")).alias("DepDelayedCount"),
        sum(expr("CASE WHEN IsArrDelayed = 'YES' THEN 1 ELSE 0 END")).alias("ArrDelayedCount")
       ). \
    orderBy("FlightDate"). \
    show(31)

+----------+-----------+---------------+---------------+
|FlightDate|FlightCount|DepDelayedCount|ArrDelayedCount|
+----------+-----------+---------------+---------------+
|2008-01-01|      19175|          11053|          11725|
|2008-01-02|      20953|          13805|          14260|
|2008-01-03|      20937|          12294|          12488|
|2008-01-04|      20929|          10175|          10593|
|2008-01-05|      18066|           9507|           9801|
|2008-01-06|      19893|          11225|          11388|
|2008-01-07|      20341|           8701|           9262|
|2008-01-08|      19603|           7946|           9401|
|2008-01-09|      19820|           6339|           7234|
|2008-01-10|      20297|           7374|           8906|
|2008-01-11|      20349|           7779|           8688|
|2008-01-12|      16572|           4128|           4304|
|2008-01-13|      18946|           6993|           7832|
|2008-01-14|      20176|           6830|           8013|
|2008-01-15|      19503|       

### Sorting Data in descending order by count

In [44]:
from pyspark.sql.functions import lit, concat, lpad, sum, expr, col
airlines. \
    groupBy(concat("Year", lit("-"), 
                   lpad("Month", 2, "0"), lit("-"), 
                   lpad("DayOfMonth", 2, "0")).
            alias("FlightDate")). \
    agg(count(lit(1)).alias("FlightCount"),
        sum(expr("CASE WHEN IsDepDelayed = 'YES' THEN 1 ELSE 0 END")).alias("DepDelayedCount"),
        sum(expr("CASE WHEN IsArrDelayed = 'YES' THEN 1 ELSE 0 END")).alias("ArrDelayedCount")
       ). \
    orderBy(col("FlightCount").desc()). \
    show()

+----------+-----------+---------------+---------------+
|FlightDate|FlightCount|DepDelayedCount|ArrDelayedCount|
+----------+-----------+---------------+---------------+
|2008-01-02|      20953|          13805|          14260|
|2008-01-03|      20937|          12294|          12488|
|2008-01-04|      20929|          10175|          10593|
|2008-01-11|      20349|           7779|           8688|
|2008-01-18|      20347|          10268|          11090|
|2008-01-07|      20341|           8701|           9262|
|2008-01-25|      20313|           9352|          11005|
|2008-01-10|      20297|           7374|           8906|
|2008-01-17|      20273|          10507|          12101|
|2008-01-31|      20260|          10208|          12385|
|2008-01-24|      20257|           8434|          10294|
|2008-01-14|      20176|           6830|           8013|
|2008-01-28|      20147|           8234|           9667|
|2008-01-21|      20133|          10530|          11507|
|2008-01-06|      19893|       

## Solutions - Problem 3
Get all the flights which are departed late but arrived early (**IsArrDelayed is NO**).
* Output should contain - **FlightCRSDepTime**, **UniqueCarrier**, **FlightNum**, **Origin**, **Dest**, **DepDelay**, **ArrDelay**
* **FlightCRSDepTime** need to be computed using **Year**, **Month**, **DayOfMonth**, **CRSDepTime**
* **FlightCRSDepTime** should be displayed using **YYYY-MM-dd HH:mm** format.
* Output should be sorted by **FlightCRSDepTime** and then by the difference between **DepDelay** and **ArrDelay**
* Also get the count of such flights


In [45]:
airlines.select('Year', 'Month', 'DayOfMonth', 'CRSDepTime').show()

+----+-----+----------+----------+
|Year|Month|DayOfMonth|CRSDepTime|
+----+-----+----------+----------+
|2008|    1|        16|      1735|
|2008|    1|        17|      1701|
|2008|    1|        17|      1225|
|2008|    1|        17|      1530|
|2008|    1|        17|      1205|
|2008|    1|        18|      1150|
|2008|    1|        18|      1009|
|2008|    1|        19|       835|
|2008|    1|        20|      1935|
|2008|    1|        20|       830|
|2008|    1|        21|      1640|
|2008|    1|        21|      1204|
|2008|    1|        21|      1935|
|2008|    1|        21|      1830|
|2008|    1|        21|       700|
|2008|    1|        22|      1910|
|2008|    1|        22|      1320|
|2008|    1|        23|       908|
|2008|    1|        23|      1252|
|2008|    1|        23|       635|
+----+-----+----------+----------+
only showing top 20 rows



In [46]:
l = [(2008, 1, 23, 700),
     (2008, 1, 10, 1855),
    ]

In [47]:
df = spark.createDataFrame(l, "Year INT, Month INT, DayOfMonth INT, DepTime INT")
df.show()

+----+-----+----------+-------+
|Year|Month|DayOfMonth|DepTime|
+----+-----+----------+-------+
|2008|    1|        23|    700|
|2008|    1|        10|   1855|
+----+-----+----------+-------+



In [48]:
from pyspark.sql.functions import substring
df.select(substring(col('DepTime'), -2, 2)). \
    show()

+-------------------------+
|substring(DepTime, -2, 2)|
+-------------------------+
|                       00|
|                       55|
+-------------------------+



In [49]:
df.select("DepTime", date_format(lpad('DepTime', 4, "0"), 'HH:mm')).show()

+-------+---------------------------------------+
|DepTime|date_format(lpad(DepTime, 4, 0), HH:mm)|
+-------+---------------------------------------+
|    700|                                  00:00|
|   1855|                                  00:00|
+-------+---------------------------------------+



In [50]:
help(substring)

Help on function substring in module pyspark.sql.functions:

substring(str, pos, len)
    Substring starts at `pos` and is of length `len` when str is String type or
    returns the slice of byte array that starts at `pos` in byte and is of length `len`
    when str is Binary type.
    
    .. note:: The position is not zero based, but 1 based index.
    
    >>> df = spark.createDataFrame([('abcd',)], ['s',])
    >>> df.select(substring(df.s, 1, 2).alias('s')).collect()
    [Row(s='ab')]
    
    .. versionadded:: 1.5



In [51]:
df.select(substring(col('DepTime'), 1, length(col('DepTime').cast('string')))). \
    show()

NameError: name 'length' is not defined

In [52]:
from pyspark.sql.functions import lit, col, concat, lpad, sum, expr

flightsFiltered = airlines. \
    filter("IsDepDelayed = 'YES' AND IsArrDelayed = 'NO'"). \
    select(concat("Year", lit("-"), 
                  lpad("Month", 2, "0"), lit("-"), 
                  lpad("DayOfMonth", 2, "0"), lit(" "),
                  lpad("CRSDepTime", 4, "0")
                 ).alias("FlightCRSDepTime"),
           "UniqueCarrier", "FlightNum", "Origin", 
           "Dest", "DepDelay", "ArrDelay"
          ). \
    orderBy("FlightCRSDepTime", col("DepDelay") - col("ArrDelay")). \
    show()

+----------------+-------------+---------+------+----+--------+--------+
|FlightCRSDepTime|UniqueCarrier|FlightNum|Origin|Dest|DepDelay|ArrDelay|
+----------------+-------------+---------+------+----+--------+--------+
| 2008-01-01 0055|           DL|      560|   LAX| ATL|      15|      -4|
| 2008-01-01 0100|           AA|     2466|   SFO| DFW|       2|      -7|
| 2008-01-01 0300|           CO|      488|   SJU| EWR|       3|     -12|
| 2008-01-01 0400|           B6|      724|   BQN| MCO|       4|     -23|
| 2008-01-01 0515|           XE|     2140|   SLC| IAH|       1|     -10|
| 2008-01-01 0525|           XE|     2429|   MSP| IAH|       1|      -4|
| 2008-01-01 0530|           XE|     2785|   BTR| IAH|       6|       0|
| 2008-01-01 0540|           OO|     5793|   CIC| SFO|       1|      -4|
| 2008-01-01 0540|           UA|      422|   SEA| DEN|       4|     -10|
| 2008-01-01 0600|           DL|      970|   FLL| ATL|       2|       0|
| 2008-01-01 0600|           AA|     1734|   SAT| D

### Getting Count

In [53]:
from pyspark.sql.functions import lit, col, concat, lpad, sum, expr

flightsFiltered = airlines. \
    filter("IsDepDelayed = 'YES' AND IsArrDelayed = 'NO'"). \
    select(concat("Year", lit("-"), 
                  lpad("Month", 2, "0"), lit("-"), 
                  lpad("DayOfMonth", 2, "0"), lit(" "),
                  lpad("CRSDepTime", 4, "0")
                 ).alias("FlightCRSDepTime"),
           "UniqueCarrier", "FlightNum", "Origin", 
           "Dest", "DepDelay", "ArrDelay"
          ). \
    count()

flightsFiltered

54233